# From Amazon Exercise
https://docs.aws.amazon.com/vpc/latest/userguide/vpc-subnets-commands-example.html

# boto documentation for EC2
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html?highlight=routetable


In [14]:
import boto3

In [48]:
ec2 = boto3.resource('ec2', region_name='us-west-2')
#client = boto3.client('ec2', region_name='us-west-2')

aws ec2 create-vpc --cidr-block 10.0.0.0/16 --query Vpc.VpcId --output text

In [49]:
vpc = ec2.create_vpc(CidrBlock='10.0.0.0/16')

In [50]:
vpc.id

'vpc-0964b75769c5a41a1'

# Creating 2 subnets in the VPC created above
- aws ec2 create-subnet --vpc-id vpc-2f09a348 --cidr-block 10.0.1.0/24
- aws ec2 create-subnet --vpc-id vpc-2f09a348 --cidr-block 10.0.0.0/24


In [52]:
subnet_0 = ec2.create_subnet(VpcId=vpc.id, CidrBlock='10.0.0.0/24', AvailabilityZone='us-west-2a')
subnet_0

ec2.Subnet(id='subnet-0e97120499befd735')

In [55]:
subnet_1 = ec2.create_subnet(VpcId=vpc.id, CidrBlock='10.0.1.0/24', AvailabilityZone='us-west-2a')
subnet_1.id

'subnet-0f740276b448a578d'

# Creating an internet gateway to allow external traffic
aws ec2 create-internet-gateway --query InternetGateway.InternetGatewayId --output text

In [56]:
gw = ec2.create_internet_gateway()
gw
#gw = ec2.InternetGateway('igw-0656a5e09e0ffd725')

ec2.InternetGateway(id='igw-0a281624a2ebc8c00')

# Attach Gateway to VPC
aws ec2 attach-internet-gateway --vpc-id vpc-2f09a348 --internet-gateway-id igw-1ff7a07b

In [57]:
response = gw.attach_to_vpc(VpcId = vpc.id)
response

{'ResponseMetadata': {'RequestId': 'd83f8936-0d40-466d-bfef-a6052d54a4d2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd83f8936-0d40-466d-bfef-a6052d54a4d2',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '243',
   'date': 'Sat, 11 Feb 2023 12:43:51 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

# Create a route table "In a VPC" to allow all internet traffic to a gateway and then to a subnet 
aws ec2 create-route-table --vpc-id vpc-2f09a348 --query RouteTable.RouteTableId --output text


In [58]:
route_table = ec2.create_route_table(VpcId=vpc.id)
route_table

ec2.RouteTable(id='rtb-0b749dd33dbf37a50')

# Create a route that points all traffic to the internet gateway 
aws ec2 create-route --route-table-id rtb-c1c8faa6 --destination-cidr-block 0.0.0.0/0 --gateway-id igw-1ff7a07b

In [59]:
route = route_table.create_route(DestinationCidrBlock='0.0.0.0/0', GatewayId=gw.id)
route

ec2.Route(route_table_id='rtb-0b749dd33dbf37a50', destination_cidr_block='0.0.0.0/0')

# Associsate route to one of the subnets created above (subnet_1)
aws ec2 associate-route-table  --subnet-id subnet-b46032ec --route-table-id rtb-c1c8faa6

In [60]:
assoc = route_table.associate_with_subnet(SubnetId=subnet_1.id)
assoc

ec2.RouteTableAssociation(id='rtbassoc-0deae5cb11f1c00d7')

You can modify the public IP addressing behavior of your subnet so that an instance launched into the subnet automatically receives a public IP address using the following modify-subnet-attribute command

In [61]:
response = client.modify_subnet_attribute(
    MapPublicIpOnLaunch={
        'Value': True,
    },
    SubnetId=subnet_1.id,
)

In [62]:
response

{'ResponseMetadata': {'RequestId': '159b9c6d-0fd5-45fc-841f-b4957c4786d3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '159b9c6d-0fd5-45fc-841f-b4957c4786d3',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '243',
   'date': 'Sat, 11 Feb 2023 12:45:29 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

# Launching an instance into the subnet

## Create a keypair

In [63]:
# Run the following commands on command line 
# aws ec2 create-key-pair --key-name MyKeyPair --query "KeyMaterial" --output text > MyKeyPair.pem
# chmod 400 MyKeyPair.pem

## Create a security group to allow ssh access 

aws ec2 create-security-group --group-name SSHAccess --description "Security group for SSH access" --vpc-id vpc-2f09a348

In [64]:
sg = ec2.create_security_group(Description='Security group for SSH access', GroupName='SSHAccess', VpcId=vpc.id)

In [65]:
sg

ec2.SecurityGroup(id='sg-0bf944ca57872c10e')

## Authorize port 22 (ssh) access to the security group 

aws ec2 authorize-security-group-ingress --group-id sg-0b24589f6b07e54ac --protocol tcp --port 22 --cidr 0.0.0.0/0

In [66]:
response = sg.authorize_ingress(IpProtocol='tcp', FromPort=22, ToPort=22, CidrIp='0.0.0.0/0')

## Launch an Ec2 instance
aws ec2 run-instances --image-id ami-06e85d4c3149db26a --count 1 --instance-type t2.micro --key-name MyKeyPair --security-group-ids sg-0b24589f6b07e54ac --subnet-id subnet-0bb7ea5088612392d

### Finding a Linux AMI 
https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/finding-an-ami.html

In [71]:
#instance = client.run_instances(ImageId='ami-06e85d4c3149db26a', InstanceType='t2.micro', KeyName='MyKeyPair', 
#                                SecurityGroups= [sg.id], SubnetI#d=subnet_1.id, 
#                               MinCount=1, MaxCount=1)

In [72]:
# aws ec2 describe-instances --instance-id i-0cc884253c909e121 --query "Reservations[*].Instances[*].{State:State.Name,Address:PublicIpAddress}"